## **Neural Networks Classification**

* If the output can be either 0 or 1, we call it Binary Classification.
* If there can multiple outputs, we call it Multi-class classification.

In [1]:
import tensorflow as tf
print(tf.__version__)

2.19.0


In [ ]:
from sklearn.datasets import make_circles

n_samples =1000

X, y = make_circles(n_samples, noise=0.03, random_state=42)

In [4]:
X

array([[ 0.75424625,  0.23148074],
       [-0.75615888,  0.15325888],
       [-0.81539193,  0.17328203],
       ...,
       [-0.13690036, -0.81001183],
       [ 0.67036156, -0.76750154],
       [ 0.28105665,  0.96382443]])

In [6]:
y[:20]

array([1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0])

In [8]:
import pandas as pd

circles = pd.DataFrame(
                      {'X0': X[:, 0], 'X1': X[:, 1], 'label': y}
                      )
circles.head()

,X0,X1,label
0,0.754246,0.231481,1
1,-0.756159,0.153259,1
2,-0.815392,0.173282,1
3,-0.393731,0.692883,1
4,0.442208,-0.896723,0


In [10]:
circles.label.value_counts()

,count
label,
1,500
0,500
